In [4]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import math
import hashlib
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import svm,linear_model
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel,RFE,RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler,KBinsDiscretizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR,LinearSVR
from sklearn import model_selection
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from numpy import transpose
from collections.abc import Iterable
from sklearn.metrics import confusion_matrix
import os


In [5]:
cwd = os.getcwd()
print(cwd)

/Users/jessicapan/Documents/GitHub/TRPM8-bootcamp-project/4_feature_selection


In [7]:
#Put in fname the csv file of the molecular descriptors calculated for each molecule

fname = "../3_train_test_split/train_set.csv"
df = pd.read_csv("../3_train_test_split/train_set.csv")

def imputing_missing_values(data):
    return SimpleImputer(missing_values=np.NaN, strategy='mean')

imputing_missing_values(fname)

SimpleImputer()

In [11]:
def write_ft(titles,features):
    with open('ft_selected','w') as f:
        for ft in features:
            f.write(titles[ft]+'\n')

In [ ]:
target = 1010
def read_data(fname,target,excluded=[]):
    classification,titles,data = [],[],[]
    
    with open (fname, 'r') as ds:
        titles=ds.readline().strip().split(',')
        target=len(titles)+target if target<0 else target
        titles=[titles[i] for i in range(len(titles)) if i not in excluded and i != target]
        for x in ds:
            line=x.strip().split(',')
            classification.append(float(line[target]))
            data.append([float(line[i]) if len(line[i])>0 else np.NaN for i in range(len(line)) if i not in excluded and i != target ]) 
    return np.array(data),np.array(classification),titles

read_data(fname,target,excluded=[])

(array([[-1.54625991e+00, -7.43806211e-01, -9.40078681e-01, ...,
         -1.88487169e+00, -1.51316101e+00,  1.37957079e+01],
        [ 9.43718034e-01,  1.90366103e-01,  1.58646673e-01, ...,
          1.50430227e+00,  1.67163265e+00,  1.59696756e+01],
        [-1.44770740e-02, -1.03467341e+00, -1.01412805e+00, ...,
         -1.42598839e-01, -5.61326779e-02,  1.61180957e+01],
        ...,
        [-6.81473458e-01,  2.28875998e+00,  2.00708531e+00, ...,
         -7.44726101e-01,  2.58481679e-01,  1.15129255e+01],
        [-5.96876682e-01, -3.09465048e-01, -5.27212115e-01, ...,
          3.94798579e-01, -3.25765521e-01,  1.47596865e+01],
        [-8.69012835e-01, -4.15525424e-01, -4.19636413e-01, ...,
         -7.19501641e-01, -1.02012076e+00,  1.24143276e+01]]),
 array([ 1.78540179, -0.13192624,  1.78540179, -0.13192624, -0.13192624,
        -2.04925427, -0.13192624, -0.13192624, -2.04925427,  1.78540179,
        -0.13192624, -0.13192624, -0.13192624, -0.13192624, -0.13192624,
        -0

In [16]:
target = len(df.columns)
target

1010